In [ ]:
# 可刪掉的block
"/Users/cfh00892302/Desktop/myWorkspace/Chinese-Verdict-NLP/dev/label_output_new/merged_data.json"

1. 把資料合併 #暫時不用做這步驟

In [2]:
from information_extraction.utils.base_utils import merge_json
all_json_file_locate_folder = "./dev/label_output_raw"
merged_json_output_folder = "./dev/label_output_new"
merge_json(all_json_file_locate_folder, merged_json_output_folder)

[2023-05-26 14:19:18,522] [    INFO] - Find the following json file: ['個理_文本分類_119.json', '商險_文本分類_25.json'].
[2023-05-26 14:19:18,523] [    INFO] - Merging the file 個理_文本分類_119.json...
[2023-05-26 14:19:18,536] [    INFO] - Length of 個理_文本分類_119.json is 119.
[2023-05-26 14:19:18,538] [    INFO] - Merging the file 商險_文本分類_25.json...
[2023-05-26 14:19:18,543] [    INFO] - Length of 商險_文本分類_25.json is 25.
[2023-05-26 14:19:18,561] [    INFO] - Successful merge all file (len=144) to ./dev/label_output_new/merged_data.json.


In [6]:
file_name = "./dev/label_output_new/merged_data.json"

2. 檢查資料正確性

In [11]:
from information_extraction.utils.base_utils import is_repeat_content_exist
result = is_repeat_content_exist(file_name)
if not result['is_repeat']:
    print("OK! No repeat.")
else:
    print("Duplicate cases happened!!!")
    

[2023-05-26 14:21:56,809] [    INFO] - All data: 144. Unique data: 144. Repeat data: 0.


OK! No repeat.


3. 資料前處理 去特殊字元

In [5]:
from information_extraction.utils.base_utils import regularize_json_file
regularize_json_file(json_file="./dev/label_output_new/merged_data.json", output_path="./information_extraction/data/first_stage_data/")

[2023-05-26 06:58:26,428] [   DEBUG] - result_index = [2107, 2113, 2764, 2767, 3122, 3128, 3132, 3135]
[2023-05-26 06:58:26,429] [   DEBUG] - result_index = [3441, 3447, 3464, 3470, 4388, 4391, 4439, 4445, 4466, 4473]
[2023-05-26 06:58:26,430] [   DEBUG] - result_index = [5796, 5802, 8332, 8335, 8476, 8479, 8541, 8547, 8586, 8589]
[2023-05-26 06:58:26,432] [   DEBUG] - result_index = [1569, 1575, 3263, 3271, 3430, 3433, 3476, 3481, 3498, 3504]
[2023-05-26 06:58:26,433] [   DEBUG] - result_index = [2179, 2183, 3217, 3223]
[2023-05-26 06:58:26,434] [   DEBUG] - result_index = [5131, 5139, 7073, 7080, 7145, 7152, 8323, 8327, 8393, 8401, 8457, 8465]
[2023-05-26 06:58:26,436] [   DEBUG] - result_index = [1742, 1749, 1849, 1856, 2837, 2840, 3258, 3265, 3390, 3397, 4141, 4148, 4165, 4172]
[2023-05-26 06:58:26,437] [   DEBUG] - result_index = [1224, 1230, 1365, 1371, 2227, 2230, 2279, 2285, 2300, 2307]
[2023-05-26 06:58:26,438] [   DEBUG] - result_index = [2328, 2336, 2543, 2551, 3284, 3291, 3

3. Split into train/eval/test

In [9]:
!python ./information_extraction/utils/split_labelstudio.py \
    --labelstudio_file ./dev/label_output_new/regularized_data.json \
    --save_dir information_extraction/data/first_stage_data/ \
    --split_ratio 0.8 0.1 0.1 

/home/ubuntu/work/venv_pytorch/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[2023-05-26 07:00:54,705] [   DEBUG] - This dir = /home/ubuntu/work/Chinese-Verdict-NLP
[2023-05-26 07:00:54,706] [    INFO] - Converting regularized_data.json into information_extraction/first_stage_data/...
[2023-05-26 07:00:54,706] [ WARNING] - information_extraction/first_stage_data/ not found. Automatically making a directory...
[2023-05-26 07:00:54,737] [   DEBUG] - in do_split, len(dataset)=290
[2023-05-26 07:00:54,738] [   DEBUG] - In convert_format, data len = 232
[2023-05-26 07:00:54,739] [   DEBUG] - In convert_format, data len = 29
[2023-05-26 07:00:54,739] [   DEBUG] - In convert_format, data len = 29
[2023-05-26 07:00:54,739] [   DEBUG] - len(training_data.txt) = 696
[2023-05-26 07:00:54,803] [   DEBUG] - len(eval_data.txt) = 87
[2023-05-26 07:00:54,811] [   DEBUG] - len(testing_dat